In [44]:
import pandas as pd
import numpy as np
import imaplib
import email
import quopri
import re
import html
from bs4 import BeautifulSoup
import requests

In [2]:
username = 'prospectstudent@colby.edu'
password = 'Student.2017'

# username = 'haohaom2@gmail.com'
# password = '62947394'

### Accessing the email

In [3]:
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(username, password)
mail.list()
# Out: list of "folders" aka labels in gmail.
mail.select("inbox") # connect to inbox.

('OK', [b'4000'])

In [4]:
result, data = mail.uid('search', None, "ALL") # search and return uids instead

first_email_id = data[0].split()[0]
latest_email_uid = data[0].split()[-1]

ids = data[0].split()

In [5]:
print('first email uid ', first_email_id)
print('latest_email_uid ', latest_email_uid)

first email uid  b'4009'
latest_email_uid  b'12161'


### Reading the email message

In [6]:
result, data = mail.uid('fetch', ids[-3], '(RFC822)')
raw_email = str(data[0][1], 'utf-8')
email_message = email.message_from_string(raw_email)
print(email_message['subject'])

Google Alert - "Rebecca Corbett"


In [7]:
def get_text_from_email(email_msg):
    if email_msg.is_multipart():
        s = ''
        for payload in email_msg.get_payload():
            s += payload.get_payload()
    else:
        s = b.get_payload()
    
    return s

In [8]:
text = get_text_from_email(email_message).replace('\r\n', '')

In [9]:
text = get_text_from_email(email_message)
# text

In [20]:
text = html.unescape(text)
text

'PT09IE5ld3MgLSAxIG5ldyByZXN1bHQgZm9yIFsiUmViZWNjYSBDb3JiZXR0Il0gPT09DQoNClB1\r\nbGl0emVyLXdpbm5pbmcgTmV3IFlvcmsgVGltZXMgZWRpdG9yIHJlY291bnRzIGh1bWJsZSBiZWdp\r\nbm5pbmdzIGluDQpXYXRlcnZpbGxlDQpLZW5uZWJlYyBKb3VybmFsICYgTW9ybmluZyBTZW50aW5l\r\nbA0KUmViZWNjYSBDb3JiZXR0IGlzIHRoZSBhc3Npc3RhbnQgbWFuYWdpbmcgZWRpdG9yIG9mIHRo\r\nZSBtb3N0IHByZXN0aWdpb3VzDQphbmQgaW5mbHVlbnRpYWwgbmV3cyBvcmdhbml6YXRpb24gaW4g\r\ndGhlIGNvdW50cnkuIFNoZSBsZWQgYSBkdW8gb2YNCmludmVzdGlnYXRpdmUgcmVwb3J0ZXJzIGlu\r\nIHRoZWlyIHB1cnN1aXQgdG8gZXhwb3NlIHRoZSBkZWNhZGVzIG9mIHNleHVhbA0KaGFyYXNzbWVu\r\ndCBwZXJwZXRyYXRlZCBieSBIYXJ2ZXkgV2VpbnN0ZWluLCBhIHBvd2VyZnVsIEhvbGx5d29vZCBw\r\ncm9kdWNlciwNCndoaWNoIGluIHR1cm4gLi4uDQo8aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS91cmw/\r\ncmN0PWomc2E9dCZ1cmw9aHR0cHM6Ly93d3cuY2VudHJhbG1haW5lLmNvbS8yMDE4LzA0LzI0L25l\r\ndy15b3JrLXRpbWVzLWVkaXRvci1yZWNvdW50cy1odW1ibGUtYmVnaW5uaW5ncy1pbi13YXRlcnZp\r\nbGxlLyZjdD1nYSZjZD1DQUVZQUNvVU1UYzJNamsyT0RVMU1EVTVOVFV6TlRZNU1Ua3lHbUZtWlRS\r\naE1tTTNNR1kyWm1VNU1HWTZZMjl0T21WdU9sVlQ

In [11]:
orig_mail = "https://www.google.com/url?rct=3Dj\\u0026sa=3Dt\\u0026url=3Dhtt=\r\nps://www.stcatharinesstandard.ca/news-story/8403018-brock-university-presid=\r\nent-joins-summer-games-board/\\u0026ct=3Dga\\u0026cd=3DCAEYACoUMTA4NzA5ODA0Nj=\r\nE1MzEwMzA5NzkyGjdiZGFiOGQwYWNiM2Q4N2E6Y29tOmVuOlVT\\u0026usg=3DAFQjCNG7yoIIf=\r\ngYAYeZfImBBSxksPFzR0w"

In [12]:
quopri.decodestring(orig_mail)

b'https://www.google.com/url?rct=j\\u0026sa=t\\u0026url=https://www.stcatharinesstandard.ca/news-story/8403018-brock-university-president-joins-summer-games-board/\\u0026ct=ga\\u0026cd=CAEYACoUMTA4NzA5ODA0NjE1MzEwMzA5NzkyGjdiZGFiOGQwYWNiM2Q4N2E6Y29tOmVuOlVT\\u0026usg=AFQjCNG7yoIIfgYAYeZfImBBSxksPFzR0w'

### Using Regex to get the links

In [59]:

m = re.findall('alerts/feeds\S+', text)
url = ('https://www.google.com/' + m[0]).replace('\"', '')
url

'https://www.google.com/alerts/feeds/16218955178302834825/12493788028705499880'

In [72]:
r  = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [88]:
# use &ct as a stopper

web_lists = soup.find_all('link')[1:]
for web in web_lists:
    print(web)

<link href="https://www.google.com/url?rct=j&amp;sa=t&amp;url=https://www.centralmaine.com/2018/04/24/new-york-times-editor-recounts-humble-beginnings-in-waterville/&amp;ct=ga&amp;cd=CAIyGmFmZTRhMmM3MGY2ZmU5MGY6Y29tOmVuOlVT&amp;usg=AFQjCNGXH5-DBXWWt65KErDH2mZAb-KuvA"/>


In [84]:
re.findall('url=\\S+', web_lists[0])

TypeError: expected string or bytes-like object

In [89]:
def get_first_text_block(email_message_instance):
    maintype = email_message_instance.get_content_maintype()
    if maintype == 'multipart':
        for part in email_message_instance.get_payload():
            if part.get_content_maintype() == 'text':
                return part.get_payload()
    elif maintype == 'text':
        return email_message_instance.get_payload()

In [107]:
# get_first_text_block(email_message)

#### using mail.search()

In [70]:
type, data = mail.search(None, 'ALL')
mail_ids = data[0].decode()
id_list = mail_ids.split()

In [71]:
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])

In [89]:
typ, data = mail.fetch('4', '(RFC822)' )

In [91]:
for response_part in data:
    if isinstance(response_part, tuple):
        msg = email.message_from_string(str(response_part[1]))
        email_subject = msg['subject']
        email_from = msg['from']
        print( 'From : ' + str(email_from) + '\n')
        print ('Subject : ' + str(email_subject) + '\n')

From : None

Subject : None



In [97]:
raw_email = data[0][1]
email_message = email.message_from_string(str(raw_email))
print(email_message['To'])

None


In [84]:
for response_part in data:
    if isinstance(response_part, tuple):
        print(type(response_part[1]))

<class 'bytes'>


In [65]:
import smtplib
import time
import imaplib
import email
def read_email_from_gmail(user, pw):
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(user, pw)
        mail.select('inbox')

        typ, data = mail.search(None, 'ALL')
        mail_ids = data[0]

        id_list = mail_ids.split()   
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])


        for i in range(latest_email_id,first_email_id, -1):
            typ, data = mail.fetch(str(i), '(RFC822)' )

            for response_part in data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_string(response_part[1])
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print( 'From : ' + email_from + '\n')
                    print ('Subject : ' + email_subject + '\n')

    except Exception as e:
        print (e)

In [66]:
read_email_from_gmail(username, password)

initial_value must be str or None, not bytes
